### Librairies

In [1]:
# Librairies générales
import sys
import cv2
import time
import os
import numpy as np

### Afficher texte sur image

In [2]:
def draw_str(dst,x, y, s):
    cv2.putText(dst, s,(x+1,y+1), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 0, 0),
                thickness=2, lineType=cv2.LINE_AA)
    cv2.putText(dst, s,(x,y), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255),
                lineType=cv2.LINE_AA)

### Classe Descripteur_Image

In [3]:
# Classe Descripteur_Image
# Pour une image, contient les informations necessaires à la reconnaissance de l'objet
class Descripteur_Image(object):
    
    def __init__(self, nom_fichier, shape, image_couleur, points_cles, descripteurs):
        
        # Nom du fichier
        self.nom_fichier = nom_fichier
        
        # Format de l'image
        self.shape = shape
        
        # Forme couleur de l'image
        self.image_couleur = image_couleur
        
        # Les points clés de l'image
        self.points_cles = points_cles
        
        # Descripteurs des points clés
        self.descripteurs = descripteurs
        
         # Association de l'image avec l'image de la video
        self.reconnaissanceVideo = []
        
        # Association de la vidéo avec l'objet de la base de données
        self.reconnaissanceBaseDonnee = []
        
    # Nettoyage des associations  effectuée pour un nouveau calcul
    def nettoyageAssociation(self):
        self.reconnaissanceVideo = []
        self.reconnaissanceBaseDonnee = []

### Chargement des points depuis la base de données

In [4]:
# Pour chaque images de la base de donnée on charge les points clés
def chargementPointsClesBaseDeDonnee():
    # Retourne une liste d'"ImageFeature" contenant les descripteurs des Points clés
    # des images de la base de donnée
    
    res = list()
    
    # On choisis le nombre de points clés par image (influe sur la fluidité de l'algorithme)
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=300)
    
    for image in os.listdir("BASE_DONNEES"):
        
        # Chargement des images par openCV
        image_couleur = cv2.imread("BASE_DONNEES/" + str(image))
        
        # On change l'image couleur en image de niveau de gris
        image_actuelle = cv2.cvtColor(image_couleur, cv2.COLOR_BGR2GRAY)
        
        # Obtention des points clés et des descripteurs de l'image
        points_cles, descripteurs = sift.detectAndCompute(image_actuelle, None)
        
        # Features SIFT
        res.append(Descripteur_Image(image, image_actuelle.shape, image_couleur, points_cles, descripteurs))
        
    return res

### Associations entres les images de la BD et l'image video

In [5]:
# Calcul les associations de l'image video, avec les images de la base de données.
# Parametre d'entreée : 
# - Données des images de la BD 
# - Descripteurs de l'image video
# - Points clés de l'image video
def trouverAssociation(data, descripteurs, points_cles):
    
    for img in data:
        img.nettoyageAssociation()
        for i in range(len(descripteurs)):
            
            # Norme de la difference entre le descripteur avec tous les autres descripteurs.
            distanceAvecVideo = np.linalg.norm(descripteurs[i] - img.descripteurs, axis=-1)
            
            # Descripteur ayant une distance minimale par rapport au descripteur étudié
            pointCandidat = distanceAvecVideo.argmin() 
            
            # L'association est elle bidirectionnelle ?
            distanceAvecBD = np.linalg.norm(img.descripteurs[pointCandidat] - descripteurs,axis=-1)
            candidatImageVideo = distanceAvecBD.argmin()
                
            # Si l'association est bonne on fait l'affectation
            if (i == candidatImageVideo):
                img.reconnaissanceVideo.append(points_cles[i].pt)
                img.reconnaissanceBaseDonnee.append(img.points_cles[pointCandidat].pt)
                
        # Conversion en tableau numpy
        img.reconnaissanceVideo = np.array(img.reconnaissanceVideo)
        img.reconnaissanceBaseDonnee = np.array(img.reconnaissanceBaseDonnee)
    return data

### Trouver l'image de la base de donnée qui correspond le mieux

In [6]:
# Calcul l'image ayant une meilleure correspondance selon la taille minimale du cluster
def calculerMeilleureAssociation(data, nb_Erreur_Projection, minInliers):

    bestIndex = None
    bestMask = None
    numInliers = 0
    # Poour chaque image de la BD
    for index, imgWithMatching in enumerate(data):
        # donne la perspective entre deux plans, et evalue le nombre d'erreur par la methode RANSAC
        _, mask = cv2.findHomography(imgWithMatching.reconnaissanceBaseDonnee, 
                                     imgWithMatching.reconnaissanceVideo, cv2.RANSAC, nb_Erreur_Projection)
        if not mask is None:
            # On verifie que la taille est à la fois plus grande que le minimum requis 
            # et que l'image precedente.
            countNonZero = np.count_nonzero(mask)
            if (countNonZero >= minInliers and countNonZero > numInliers):
                numInliers = countNonZero
                bestIndex = index
                bestMask = (mask >= 1).reshape(-1)
    # Si une image valide les criteres, on retourne celle-ci avec le cluster qui l'a validé
    if not bestIndex is None:
        bestImage = data[bestIndex]
        inliersWebCam = bestImage.reconnaissanceVideo[bestMask]
        inliersDataBase = bestImage.reconnaissanceBaseDonnee[bestMask]
        return bestImage, inliersWebCam, inliersDataBase
    return None, None, None

### Matrice de correspondance + rectangle encerclant l'objet + affichage de l'objet correspondant

In [7]:
# On calcule la matrice d'affinité, on affiche un rectangle entourant l'objet 
# et on ouvre une fenetre montrant l'objet reconnu.

def matriceAffinite_et_dessin(image_meilleure_correspondance, clustersBaseDonnees, clustersVideo, image_sortie):
    
    # Matrice affinités
    # transformation affine optimale entre des ensembles de points 2D
    A = cv2.estimateRigidTransform(clustersBaseDonnees, clustersVideo, fullAffine=True)
    A = np.vstack((A, [0, 0, 1]))
    
    # Points du rectangle entourant l'objet
    a = np.array([0, 0, 1], np.float)
    b = np.array([image_meilleure_correspondance.shape[1], 0, 1], np.float)
    c = np.array([image_meilleure_correspondance.shape[1], image_meilleure_correspondance.shape[0], 1], np.float)
    d = np.array([0, image_meilleure_correspondance.shape[0], 1], np.float)
    centre = np.array([float(image_meilleure_correspondance.shape[0])/2, float(image_meilleure_correspondance.shape[1])/2, 1], np.float)
       
    # Mise à l'echelle des points pour fitter avec l'echelle de l'image video
    a = np.dot(A, a)
    b = np.dot(A, b)
    c = np.dot(A, c)
    d = np.dot(A, d)
    centre = np.dot(A, centre)
    
    a_video = (int(a[0]/a[2]), int(a[1]/b[2]))
    b_video = (int(b[0]/b[2]), int(b[1]/b[2]))
    c_video = (int(c[0]/c[2]), int(c[1]/c[2]))
    d_video = (int(d[0]/d[2]), int(d[1]/d[2]))
    centre_video_x = int(centre[0]/centre[2])
    centre_video_y = int(centre[1]/centre[2])
    
    # Affichage du rectangle
    points = np.array([a_video, b_video, c_video, d_video], np.int32)
    cv2.polylines(image_sortie, np.int32([points]),1, (255,255,255), thickness=3)
    draw_str(image_sortie, centre_video_x,centre_video_y, image_meilleure_correspondance.nom_fichier.upper())
    #Se visualiza el objeto detectado en una ventana a parte
    cv2.imshow('Objet', image_meilleure_correspondance.image_couleur)


# Programme

In [ ]:
    # Affichage de la fenetre du programme
    def fenetre(*arg):
        pass
    cv2.namedWindow("Reconnaissance d'objets")
    cv2.namedWindow('Objet')
    

    # Taille cluster minimal pour reconnaitre l'objet
    cv2.createTrackbar('Taille Cluster', "Reconnaissance d'objets", 20, 50, fenetre)
    
    # Choix de l'affichage des points clé
    cv2.createTrackbar('Afficher Points cles', "Reconnaissance d'objets", 0,1, fenetre)
    
    cv2.createTrackbar('nb Erreur projection', "Reconnaissance d'objets",5,10, fenetre)
    
    vc = cv2.VideoCapture(0)
    pause = False

    # Chargement des points pour chaque image du dossier "BASE_DONNEES"
    points_BD = chargementPointsClesBaseDeDonnee()
    
    # Boucle d'execution:
    while True:
        
        if not pause:
            rval, affichage = vc.read()
        if affichage is None:
            print('End of video input')
            break

        # Initialisation du detecteur SIFT :
        detecteur = cv2.xfeatures2d.SIFT_create(nfeatures=300)
                
        # Conversion image entrée en gris:
        image_entree = cv2.cvtColor(affichage, cv2.COLOR_BGR2GRAY)
        
        # Image de sortie (pour ne pas modifier l'image d'entree)
        image_sortie = affichage.copy()
        
        # Calcul du temps et des points clés + descripteurs:
        t1 = time.time()
        points_cles, descripteurs = detecteur.detectAndCompute(image_entree, None)
        
        if len(points_BD) > 0:
            # Associations de points clés entre l'image vidéo et celles des images de la BD
            images_avec_association = trouverAssociation(points_BD, descripteurs, points_cles)    
            
            # Obtention du parametre concernant la taille minimale du cluster
            taille_minimale_cluster = int(cv2.getTrackbarPos('Taille Cluster', "Reconnaissance d'objets"))
            
            nb_Erreur_Projection = float(cv2.getTrackbarPos('nb Erreur projection', "Reconnaissance d'objets"))
            
            # Calcul de l'image de BD correspondant au mieux 
            image_meilleure_correspondance, cluster_video, cluster_bd =  calculerMeilleureAssociation(points_BD, nb_Erreur_Projection,taille_minimale_cluster)            
            
            if not image_meilleure_correspondance is None:
                # Une image correspond ->  Affichage sur l'image video de l'objet
                matriceAffinite_et_dessin(image_meilleure_correspondance, cluster_bd, cluster_video, image_sortie)
               
        t1 = 1000 * (time.time() - t1)  # Temps en milisecondes
        
        # Taille des descripteurs des points clés:
        if descripteurs is not None:
            if len(descripteurs) > 0:
                dim = len(descripteurs[0])
            else:
                dim = -1
                
        # Affichage des descripteurs et du texte de l'objet
        if (int(cv2.getTrackbarPos('Afficher Points cles', "Reconnaissance d'objets")) > 0):
            cv2.drawKeypoints(image_sortie, points_cles, image_sortie,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        draw_str(image_sortie,20, 20,"Method {0}, {1} Points cles trouves, desc. dim. = {2} ".format('SIFT', len(points_cles), dim))
        draw_str(image_sortie,20, 40, "Temps calcul (ms): {0}".format(str(t1)))
        
        # Affichage des resultats :
        cv2.imshow("Reconnaissance d'objets", image_sortie)
        ch = cv2.waitKey(5) & 0xFF
        if ch == ord(' '):  # Barre espace
            pause = not pause

    # Fermeture:
    cv2.destroyAllWindows()

Code inspiré de : https://gitlab.com/josemariasoladuran/object-recognition-opencv-python